### Import Libraries

In [31]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import pyautogui
from datetime import datetime
import win32com.client as win32

### Accessing CoinMarketCap 

    - First of all it's going to be defined some key information. 
        - wallet: it's the list of the coins that that will be sought.  
        - volume: it's the quantity of each coin. 
        - price_buy: It's the price of purchase of each coin.
        - Be free to change list items, but BE AWERE THAT THE POSITION OF EACH ITEM AT THE LISTS MUST BE THE SAME 
            -Example: wallet[1] == Xcoin, volume [1] == volume_Xcoin, price_buy[1] == price_Xcoin 
    - And then we will access www.coinmarketcap.com to get the updated price of currencies in our wallet. 

In [32]:
wallet = ['polygon','iota', 'bitcoin', 'ethereum', 'cardano', 'chiliz', 'uniswap', 'sushiswap', 'nexo', 'okb', 'solana']
volume = [12500,10000,1.5,10,5000,50000,850,1500,8000,650,420]
price_buy = [2.48,1.57,25475.44,854.87,3.78,0.15,18.99,20.47,0.32,35.71,55.63]

# These empty lists are going to be filled with the updated information
coins = []
prices = []
price_variation = []

# Accessign the website 
web = webdriver.Chrome() 
web.get('https://coinmarketcap.com/')

# This is a overview of the market. It's going to be useful for the e-mail
info = web.find_element_by_class_name('sc-1eb5slv-0').text
print(info)

# Now it will be accessed the page of each coin to getter the information that is needed 
for coin in wallet:
    web.get(f'https://coinmarketcap.com/currencies/{coin}/')
    price = web.find_element_by_class_name('priceValue').text
    price_var_24 = web.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div/div[1]/div[2]/div[2]/div/div[1]/table/tbody/tr[2]/td/span').text
    print(coin, price, price_var_24)
    coins.append(coin)
    prices.append(price)
    price_variation.append(price_var_24)

# All the information gathered is printed below 

The global crypto market cap is $2.22T, a 1.58% increase over the last day.
polygon $1.44 $-0.02337
iota $1.42 $0.2872
bitcoin $49,494.10 $655.88
ethereum $3,792.83 $82.93
cardano $2.97 $0.07477
chiliz $0.3809 $-0.004618
uniswap $29.94 $-0.1622
sushiswap $13.09 $-0.1386
nexo $1.95 $0.02956
okb $21.79 $-0.5536
solana $125.32 $9.67


### Cleaning our data 
    - After we got the info from CoinMarketCap it's needed to clean our data to make the report 

In [33]:
# First of all it will be created a Pandas DataFrame
wallet_df = pd.DataFrame()

# Since the DataFrame is created, the columns will be added, each column correspond to a list created before.
wallet_df['coins'] = coins 
wallet_df['prices'] = prices
wallet_df['price_variation'] = price_variation 
wallet_df['volume'] = volume
wallet_df['price_buy'] = price_buy

# The data collected at the website must be cleaned and suited 

# Remove '$' and ',' from price values  
wallet_df['prices'] = wallet_df['prices'].apply(lambda x: str(x).replace('$',''))
wallet_df['prices'] = wallet_df['prices'].apply(lambda x: str(x).replace(',',''))
wallet_df['price_variation'] = wallet_df['price_variation'].apply(lambda x: str(x).replace('$',''))
wallet_df['price_variation'] = wallet_df['price_variation'].apply(lambda x: str(x).replace(',',''))

# Change price values to numeric
wallet_df['prices'] = pd.to_numeric(wallet_df['prices'], errors='coerce')
wallet_df['price_variation'] = pd.to_numeric(wallet_df['price_variation'], errors='coerce')

# Formating the coins names 
wallet_df['coins'] = wallet_df['coins'].apply(lambda x: str(x).capitalize())

# Drop 'NaN' values in case that an error has occurred at the gathering of the information
wallet_df = wallet_df.dropna()

# Replace the index for the coins 
wallet_df.set_index('coins', inplace=True)

# The wallet DataFrame and it's infos are printed below 
display(wallet_df)
wallet_df.info()

,prices,price_variation,volume,price_buy
coins,,,,
Polygon,1.4400,-0.023370,12500.0,2.48
Iota,1.4200,0.287200,10000.0,1.57
Bitcoin,49494.1000,655.880000,1.5,25475.44
Ethereum,3792.8300,82.930000,10.0,854.87
Cardano,2.9700,0.074770,5000.0,3.78
Chiliz,0.3809,-0.004618,50000.0,0.15
Uniswap,29.9400,-0.162200,850.0,18.99
Sushiswap,13.0900,-0.138600,1500.0,20.47
Nexo,1.9500,0.029560,8000.0,0.32


<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, Polygon to Solana
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   prices           11 non-null     float64
 1   price_variation  11 non-null     float64
 2   volume           11 non-null     float64
 3   price_buy        11 non-null     float64
dtypes: float64(4)
memory usage: 440.0+ bytes


In [34]:
# Transform the wallet DataFrame in a Excel table to be used as database to the Power Bi report 
wallet_df.to_excel('wallet.xlsx')

### Refresh the Power Bi Report 
    - At this phase it will be used the Library Pyautogui, witch controls the mouse and the keyboard of the computer. 
    - It depends a lot of the exactly position of an element at the computer screen, so if anyone wants to use this code it's necessary to make personal adaptations.

In [35]:
# Step 1: Minimize everything that it's open on the computer to avoid missclicks
pyautogui.hotkey('winleft', 'd')

# Step2: Open the report at Power Bi Desktop 
pyautogui.hotkey('winleft', 'r')
time.sleep(0.5)
pyautogui.write(r'C:\Users\wallet.pbix') #Input your path to the file here 
time.sleep(0.5)
pyautogui.press('enter')
time.sleep(25)

#Step 3: Refresh and Save the Power Bi report. The positions are personal it requires adjusts 
pyautogui.moveTo(830, 93)
pyautogui.click()
time.sleep(6)
pyautogui.hotkey('ctrl', 's')
time.sleep(3)

#Step 4: Exporting the report as a PDF file 
pyautogui.moveTo(41, 50)
pyautogui.click()
time.sleep(2)
pyautogui.moveTo(36, 292)
pyautogui.click()
time.sleep(2)
pyautogui.moveTo(338, 195)
pyautogui.click()
time.sleep(8)

#Step 5: Saving the PDF file 
pyautogui.hotkey('ctrl', 'shift', 's')
time.sleep(3)
pyautogui.moveTo(1154, 683)
pyautogui.click()
time.sleep(2)
pyautogui.write('Wallet')
pyautogui.hotkey('ctrl', 'l')
pyautogui.write(r'C:\Users\Send PDF File') #Input your path to the file here
pyautogui.press('enter')
time.sleep(0.5)
pyautogui.hotkey('alt', 'l')
pyautogui.hotkey('alt', 'f4')
time.sleep(1)

### Sending the e-mail 

    - Now it's time to send the e-mail with the report 
    - The library used at this phase is win32com, very useful fow Windows applications 

In [2]:
# Update date 
date = datetime.now().strftime('%d/%m/%Y')

# Integration with Outlook 
outlook = win32.Dispatch('outlook.application')

# Create an e-mail
email = outlook.CreateItem(0)

# Formating the e-mail

email.To = "insert_your_email_here@gmail.com"

email.Subject = f"Wallet {date}"

#The variable info was gathered at the beginning of the script 

email.HTMLBody = f"""
<p>{info}</p> 

<p>Att,</p>

<p>Automatic Python</p>
"""
# Attach the PDF file

attach = (r'C:\Users\wallet.pdf') #Input your path to the file here 
email.Attachments.Add(attach)

# Enviar e-mail

email.Send()

print('E-mail Sent')

E-mail Sent
